In [1]:
import xlwings as xw
import pandas as pd
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']

In [3]:
xw.books

Books([<Book [cytiva.xlsm]>])

In [6]:
form = '{"meta": {"desc": "service_part_reqeust", "cols": {"fe_name": "fe_이름", "fe_initial": "trunkstock_id", "req_day": "요청일", "req_time": "요청시간", "address": "주소", "del_met": "배송방법", "is_return": "왕복배송여부", "recipient": "수령인", "is_urgent": "긴급여부", "parts": "요청파트", "del_instruction": "배송요청사항"}, "std_day": "2023-01-06"}, "data": ["홍길동", "KR_HGD", "2023-01-12", "18:00", "인천 연수구 송도동 218-3 삼성바이오로직스 5공장", "퀵", 0, "홍길동", 0, {"28999466": 2, "29159123": 1, "15921499": 1}, null]}'

In [7]:
import json

In [8]:
dict_form = json.loads(form)

In [9]:
dict_form.keys()
dict_form['meta']

{'desc': 'service_part_reqeust',
 'cols': {'fe_name': 'fe_이름',
  'fe_initial': 'trunkstock_id',
  'req_day': '요청일',
  'req_time': '요청시간',
  'address': '주소',
  'del_met': '배송방법',
  'is_return': '왕복배송여부',
  'recipient': '수령인',
  'is_urgent': '긴급여부',
  'parts': '요청파트',
  'del_instruction': '배송요청사항'},
 'std_day': '2023-01-06'}

In [10]:
dict_form['data']

['홍길동',
 'KR_HGD',
 '2023-01-12',
 '18:00',
 '인천 연수구 송도동 218-3 삼성바이오로직스 5공장',
 '퀵',
 0,
 '홍길동',
 0,
 {'28999466': 2, '29159123': 1, '15921499': 1},
 None]

- Index E2
- Printed Day	E3
- Recipient	B6
- Contact	E6
- Request Date	A10
- Address	D10
- Deilevery Instructions	A14
- parts_info	B21


In [11]:
wb_pf = xw.Book('print_form.xlsx')

In [14]:
wb_pf.sheets[0].range("A1").value

## 프린트하기
- 양식채우기

In [204]:
wb_cy
ws_svc = wb_pf.sheets['SVC']


In [205]:
from datetime import datetime

In [111]:
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [86]:
str(now.date())

'2023-01-13'

In [206]:
form_data = dict_form['data']
form_data

{'fe_name': '홍길동',
 'fe_initial': 'KR_HGD',
 'req_day': '2023-01-12',
 'req_time': '18:00',
 'address': '인천 연수구 송도동 218-3 삼성바이오로직스 5공장',
 'del_met': '퀵',
 'is_return': 0,
 'recipient': '홍길동',
 'is_urgent': 0,
 'parts': {'11000322': 2, '18117251': 1, '28404190': 1},
 'del_instruction': None,
 'contact': '010-1234-1234'}

In [215]:
list(form_data['parts'].values())

[2, 1, 1]

In [240]:

is_return = form_data['is_return']

if is_return == 0 :
    is_return = None:
else :
    is_return = '왕복'

0

In [ ]:
is_urgent = form_data['is_urgent']

if is_urgent == 0 :
    is_urgent = None:
else :
    is_urgent = '긴급'


### parts JSON
1. part_no
2. serial_no
3. qty
4. currnet_stock

In [260]:
cols = ['part_no', 'serial_no', 'qty', 'currnet_stock','transfer_to','BIN']
part_no = ['11000322','18117251','28404190']
serial_no = [None,None,None]
qty = [2,1,1]
currnet_stock = ['KR_SERV01','KR_SERV01','KR_SERV01']
transfer_to = ['KR_HGD','KR_HGD','KR_HGD']
BIN= [None,None,None]
df = pd.DataFrame([part_no,serial_no,qty,currnet_stock,transfer_to,BIN],index=cols).T
df

,part_no,serial_no,qty,currnet_stock,transfer_to,BIN
0,11000322,None,2,KR_SERV01,KR_HGD,None
1,18117251,None,1,KR_SERV01,KR_HGD,None
2,28404190,None,1,KR_SERV01,KR_HGD,None


In [261]:
data=[]
for i in range(len(part_no)):
    tmp = zip(cols,[part_no[i],serial_no[i],qty[i],currnet_stock[i],transfer_to[i],BIN[i]])
    data.append(dict(tmp))
data

[{'part_no': '11000322',
  'serial_no': None,
  'qty': 2,
  'currnet_stock': 'KR_SERV01',
  'transfer_to': 'KR_HGD',
  'BIN': None},
 {'part_no': '18117251',
  'serial_no': None,
  'qty': 1,
  'currnet_stock': 'KR_SERV01',
  'transfer_to': 'KR_HGD',
  'BIN': None},
 {'part_no': '28404190',
  'serial_no': None,
  'qty': 1,
  'currnet_stock': 'KR_SERV01',
  'transfer_to': 'KR_HGD',
  'BIN': None}]

In [276]:
json.dumps(data,ensure_ascii=False,)

'[{"part_no": "11000322", "serial_no": null, "qty": 2, "currnet_stock": "KR_SERV01", "transfer_to": "KR_HGD", "BIN": null}, {"part_no": "18117251", "serial_no": null, "qty": 1, "currnet_stock": "KR_SERV01", "transfer_to": "KR_HGD", "BIN": null}, {"part_no": "28404190", "serial_no": null, "qty": 1, "currnet_stock": "KR_SERV01", "transfer_to": "KR_HGD", "BIN": null}]'

In [262]:
cols = ['part_no', 'serial_no', 'qty', 'currnet_stock','transfer_to','BIN']

res =  {
    'meta':{
        'desc':'요청파트정보',
        'cols':{
            'part_no':'파트번호',
            'serial_no':'시리얼넘버',
            'qty':'개수',
            'currnet_stock':'현재시스템로케이션위치',
            'transfer_to':'Transfer해야할 시스템로케이션',
            'BIN':'파트물리적위치'
        }
        
    },
    'parts':data
    
}

parts = json.dumps(res,ensure_ascii=False)
json_parts = json.loads(parts)

In [263]:
df_parts = pd.DataFrame(json_parts['parts'])
df_parts.index = df_parts.index + 1

In [264]:
df_parts
## Transfer_to, BIN 컬럼이 추가적으로 필요함


,part_no,serial_no,qty,currnet_stock,transfer_to,BIN
1,11000322,None,2,KR_SERV01,KR_HGD,None
2,18117251,None,1,KR_SERV01,KR_HGD,None
3,28404190,None,1,KR_SERV01,KR_HGD,None


In [265]:
bin_loc = wb_cy.sheets['Cytiva Inventory BIN']
bin_loc

<Sheet [cytiva.xlsm]Cytiva Inventory BIN>

In [266]:
bin_last_row = bin_loc.range('A100000').end('up').row
bin_last_col = bin_loc.range('AAA'+str(bin_last_row)).end('left').column
bin_last_col

4

In [267]:
df_bin= pd.DataFrame(bin_loc.range((2,1),(bin_last_row,bin_last_col)).options(numbers=int).value)

In [268]:
dict_bin = dict(zip(df_bin[0].astype(str),df_bin[1]))

In [269]:
bin_list = []
for part_no in df_parts[df_parts.columns[0]]:
    bin_list.append(dict_bin[part_no])

In [270]:
bin_list

['2B1702', '2C1803', '2B1804']

In [271]:
df_parts['BIN'] = bin_list

In [272]:
df_parts

,part_no,serial_no,qty,currnet_stock,transfer_to,BIN
1,11000322,None,2,KR_SERV01,KR_HGD,2B1702
2,18117251,None,1,KR_SERV01,KR_HGD,2C1803
3,28404190,None,1,KR_SERV01,KR_HGD,2B1804


In [259]:
# Index
# 요청출고의 인덱스번호로결정 1 로가정
ws_svc.range('E2').value = 1
# Printed Day E3
ws_svc.range('E3').value = now
# Delievery Type E5
is_return = form_data['is_return']
if is_return == 0 :
    is_return = None
else :
    is_return = '왕복'
ws_svc.range('E5').value = is_return
ws_svc.range('E3').value = now
# Recipient B6
ws_svc.range('B6').value = form_data['recipient']
# Contact E6
ws_svc.range('E6').value = form_data['contact']
# Request day A10
ws_svc.range('A10').value  = form_data['req_day']
# request hour A11
ws_svc.range('A11').value  = form_data['req_time']
# Address D10
ws_svc.range('D10').value = form_data['address']
# Deilevery Instructions A14
ws_svc.range('A14').value = form_data['del_instruction']
# parts_info A20
ws_svc.range('A20').value = df_parts

# Depart From B41
ws_svc.range('B41').value = '서울시 강서구 하늘길 247 3층 C구역'
# TEL B42
ws_svc.range('B42').value = '02-2660-3767'
# Phone B43
ws_svc.range('B43').value = '담당자 폰번호'
# URGENT E41
is_urgent = form_data['is_urgent']
if is_urgent == 0 :
    is_urgent = None       
else :
    is_urgent = '긴급' 
ws_svc.range('E41').value = is_urgent

In [289]:
import win32com.client as cli

In [300]:
outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
inbox = outlook.GetDefaultFolder(6) # 받은편지함
msg = inbox.Items #메세지 정보
msg_counter = msg.count # 총갯수

part_request = []
for ms in inbox.Items:
    if 'SVC' in ms.Subject:
        print(ms.Subject)
        part_request.append(ms)

SVC_20230114_CCH_noturgent


In [299]:
body_exam = part_request[0].Body
body = body_exam[:body_exam.rfind('}')+1]
body
json_body = json.loads(body)

json_body.keys()

form_data= json_body['data']
df_parts = pd.DataFrame(form_data['parts'])
df_parts

,part_no,serial_no,qty,currnet_stock,transfer_to,BIN
0,11000322,None,2,KR_SERV01,KR_HGD,None
1,18117251,None,1,KR_SERV01,KR_HGD,None
2,28404190,None,1,KR_SERV01,KR_CCH,None


In [302]:
dict_data

'인천 연수구 송도동 218-3 삼성바이오로직스 5공장'

In [310]:
df_parts = pd.DataFrame(dict_data['parts'])


In [333]:
part_request[0].Subject+'_prfm'

'SVC_20230114_CCH_noturgent_prfm'

In [334]:
def print_form(subject=None):
    """
    """
       
    wb_pf = xw.Book('print_form.xlsx')
    ws_svc = wb_pf.sheets['SVC']
    
    outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
    inbox = outlook.GetDefaultFolder(6) # 받은편지함
    
    
    part_request = []
    for ms in inbox.Items:
        if subject in ms.Subject:
            part_request.append(ms)
            
    
    if 'SVC' in part_request[0].Subject:
    

        body_exam = part_request[0].Body
        body = body_exam[:body_exam.rfind('}')+1]
        body
        json_body = json.loads(body)

        json_body.keys()

        form_data= json_body['data']
        df_parts = pd.DataFrame(form_data['parts'])
        df_parts
        # Index

        ws_svc.range('B21:H40').clear_contents()


        # 요청출고의 인덱스번호로결정 1 로가정
        ws_svc.range('E2').value = 1
        # Printed Day E3
        ws_svc.range('E3').value = now
        # Delievery Type E5
        is_return = form_data['is_return']
        if is_return == 0 :
            is_return = None
        else :
            is_return = '왕복'
        ws_svc.range('E5').value = is_return
        ws_svc.range('E3').value = now
        # Recipient B6
        ws_svc.range('B6').value = form_data['recipient']
        # Contact E6
        ws_svc.range('E6').value = form_data['contact']
        # Request day A10
        ws_svc.range('A10').value  = form_data['req_day']
        # request hour A11
        ws_svc.range('A11').value  = form_data['req_time']
        # Address D10
        ws_svc.range('D10').value = form_data['address']
        # Deilevery Instructions A14
        ws_svc.range('A14').value = form_data['del_instruction']
        # parts_info A20
        ws_svc.range('A20').value = df_parts
        # Depart From B41
        ws_svc.range('B41').value = '서울시 강서구 하늘길 247 3층 C구역'
        # TEL B42
        ws_svc.range('B42').value = '02-2660-3767'
        # Phone B43
        ws_svc.range('B43').value = '담당자 폰번호'
        # URGENT E41
        is_urgent = form_data['is_urgent']
        if is_urgent == 0 :
            is_urgent = None       
        else :
            is_urgent = '긴급' 
        ws_svc.range('E41').value = is_urgent




In [322]:
print_form()